# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 24 2022 9:13AM,237933,15,PNC362817A,"Person & Covey, Inc.",Released
1,May 24 2022 9:13AM,237933,15,PNC362818A,"Person & Covey, Inc.",Released
2,May 24 2022 9:13AM,237933,15,PNC362819A,"Person & Covey, Inc.",Released
3,May 24 2022 9:13AM,237933,15,PNC362820A,"Person & Covey, Inc.",Released
4,May 24 2022 9:13AM,237933,15,PNC362821A,"Person & Covey, Inc.",Released
5,May 24 2022 9:13AM,237933,15,PNC362822A,"Person & Covey, Inc.",Released
6,May 24 2022 9:13AM,237933,15,PNC362823A,"Person & Covey, Inc.",Released
7,May 24 2022 9:13AM,237933,15,PNC362824A,"Person & Covey, Inc.",Released
8,May 24 2022 9:13AM,237933,15,PNC362825A,"Person & Covey, Inc.",Released
9,May 24 2022 9:13AM,237933,15,PNC362826A,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,237940,Released,1
43,237942,Released,4
44,237943,Released,1
45,237944,Executing,1
46,237945,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237940,NaN,NaN,1.0
237942,NaN,NaN,4.0
237943,NaN,NaN,1.0
237944,NaN,1.0,NaN
237945,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237673,0.0,0.0,1.0
237701,0.0,0.0,1.0
237815,0.0,0.0,1.0
237825,0.0,0.0,1.0
237840,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237673,0,0,1
237701,0,0,1
237815,0,0,1
237825,0,0,1
237840,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237673,0,0,1
1,237701,0,0,1
2,237815,0,0,1
3,237825,0,0,1
4,237840,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237673,,,1
1,237701,,,1
2,237815,,,1
3,237825,,,1
4,237840,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 24 2022 9:13AM,237933,15,"Person & Covey, Inc."
25,May 24 2022 9:11AM,237945,10,Beach Products Inc
27,May 24 2022 9:11AM,237944,18,Hush Hush
28,May 24 2022 9:04AM,237943,19,"GCH Granules USA, Inc."
29,May 24 2022 8:59AM,237942,10,Emerginnova
33,May 24 2022 8:50AM,237936,15,"Carwin Pharmaceutical Associates, LLC"
35,May 24 2022 8:47AM,237937,18,"Blinc International, Inc."
36,May 24 2022 8:47AM,237935,15,"Virtus Pharmaceuticals, LLC"
48,May 24 2022 8:44AM,237940,10,"Senseonics, Incorporated"
49,May 24 2022 8:42AM,237931,15,"Laser Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 24 2022 9:13AM,237933,15,"Person & Covey, Inc.",,,25
1,May 24 2022 9:11AM,237945,10,Beach Products Inc,,,2
2,May 24 2022 9:11AM,237944,18,Hush Hush,,1,
3,May 24 2022 9:04AM,237943,19,"GCH Granules USA, Inc.",,,1
4,May 24 2022 8:59AM,237942,10,Emerginnova,,,4
5,May 24 2022 8:50AM,237936,15,"Carwin Pharmaceutical Associates, LLC",,,2
6,May 24 2022 8:47AM,237937,18,"Blinc International, Inc.",,,1
7,May 24 2022 8:47AM,237935,15,"Virtus Pharmaceuticals, LLC",,,12
8,May 24 2022 8:44AM,237940,10,"Senseonics, Incorporated",,,1
9,May 24 2022 8:42AM,237931,15,"Laser Pharmaceuticals, LLC",,,14


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 24 2022 9:13AM,237933,15,"Person & Covey, Inc.",25,,
1,May 24 2022 9:11AM,237945,10,Beach Products Inc,2,,
2,May 24 2022 9:11AM,237944,18,Hush Hush,,1,
3,May 24 2022 9:04AM,237943,19,"GCH Granules USA, Inc.",1,,
4,May 24 2022 8:59AM,237942,10,Emerginnova,4,,
5,May 24 2022 8:50AM,237936,15,"Carwin Pharmaceutical Associates, LLC",2,,
6,May 24 2022 8:47AM,237937,18,"Blinc International, Inc.",1,,
7,May 24 2022 8:47AM,237935,15,"Virtus Pharmaceuticals, LLC",12,,
8,May 24 2022 8:44AM,237940,10,"Senseonics, Incorporated",1,,
9,May 24 2022 8:42AM,237931,15,"Laser Pharmaceuticals, LLC",14,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 24 2022 9:13AM,237933,15,"Person & Covey, Inc.",25,,
1,May 24 2022 9:11AM,237945,10,Beach Products Inc,2,,
2,May 24 2022 9:11AM,237944,18,Hush Hush,,1,
3,May 24 2022 9:04AM,237943,19,"GCH Granules USA, Inc.",1,,
4,May 24 2022 8:59AM,237942,10,Emerginnova,4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 24 2022 9:13AM,237933,15,"Person & Covey, Inc.",25.0,NaN,NaN
1,May 24 2022 9:11AM,237945,10,Beach Products Inc,2.0,NaN,NaN
2,May 24 2022 9:11AM,237944,18,Hush Hush,NaN,1.0,NaN
3,May 24 2022 9:04AM,237943,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
4,May 24 2022 8:59AM,237942,10,Emerginnova,4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 24 2022 9:13AM,237933,15,"Person & Covey, Inc.",25.0,0.0,0.0
1,May 24 2022 9:11AM,237945,10,Beach Products Inc,2.0,0.0,0.0
2,May 24 2022 9:11AM,237944,18,Hush Hush,0.0,1.0,0.0
3,May 24 2022 9:04AM,237943,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
4,May 24 2022 8:59AM,237942,10,Emerginnova,4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1189619,11.0,0.0,0.0
12,475723,1.0,1.0,0.0
15,1427592,114.0,0.0,0.0
16,713702,10.0,2.0,0.0
17,237924,1.0,0.0,0.0
18,1664821,25.0,1.0,0.0
19,1903309,11.0,5.0,17.0
20,1427358,94.0,14.0,0.0
21,475791,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1189619,11.0,0.0,0.0
1,12,475723,1.0,1.0,0.0
2,15,1427592,114.0,0.0,0.0
3,16,713702,10.0,2.0,0.0
4,17,237924,1.0,0.0,0.0
5,18,1664821,25.0,1.0,0.0
6,19,1903309,11.0,5.0,17.0
7,20,1427358,94.0,14.0,0.0
8,21,475791,2.0,0.0,0.0
9,22,475773,0.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,11.0,0.0,0.0
1,12,1.0,1.0,0.0
2,15,114.0,0.0,0.0
3,16,10.0,2.0,0.0
4,17,1.0,0.0,0.0
5,18,25.0,1.0,0.0
6,19,11.0,5.0,17.0
7,20,94.0,14.0,0.0
8,21,2.0,0.0,0.0
9,22,0.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,11.0
1,12,Released,1.0
2,15,Released,114.0
3,16,Released,10.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,18,19,20,21,22
Status,,,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0
Executing,0.0,1.0,0.0,2.0,0.0,1.0,5.0,14.0,0.0,2.0
Released,11.0,1.0,114.0,10.0,1.0,25.0,11.0,94.0,2.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0
1,Executing,0.0,1.0,0.0,2.0,0.0,1.0,5.0,14.0,0.0,2.0
2,Released,11.0,1.0,114.0,10.0,1.0,25.0,11.0,94.0,2.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0
1,Executing,0.0,1.0,0.0,2.0,0.0,1.0,5.0,14.0,0.0,2.0
2,Released,11.0,1.0,114.0,10.0,1.0,25.0,11.0,94.0,2.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()